# QAOA for the QAP

This code was modified from tutorials on quantum optimisation provided by Qiskit from 2020-2022. Note the licence at the end of the notebook

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
import qiskit

from qiskit.aqua.algorithms import QAOA
from qiskit.algorithms.optimizers import SPSA
from qiskit.aqua import QuantumInstance

from docplex.mp.model import Model
from qiskit_optimization import QuadraticProgram
from qiskit import IBMQ
from qiskit.optimization.applications.ising import docplex
#

#import logging
import numpy as np
import pandas as pd
import math


For new versions of Qiskit, be prepared for a deprecation warning

## Important prelim functions

In [1]:
#small sized matrices(under 10x10) (quick on all methods)
matrix_size_3 = './data/made3.csv'
matrix_size_4 = './data/made4.csv'
matrix_size_5 = './data/made5.csv'
matrix_size_6 = './data/made6.csv'
matrix_size_7 = './data/made7.csv'
matrix_size_8 = './data/made8.csv'
matrix_size_9 = './data/made9.csv'

matrixMade = ['./data/made4.csv', 
              './data/made5.csv', 
              './data/made6.csv', 
              './data/made7.csv', 
              './data/made8.csv', 
              './data/made9.csv']

In [2]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [3]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length)).astype(int)
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Feasibility functions

In [4]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n)).astype(int)
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [5]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

## IBM device loading

In [6]:
from qiskit import IBMQ
#IBMQ.save_account('INSERT KEY IF NECESSARY')

In [ ]:
from qiskit import IBMQ

# Load account from disk
IBMQ.load_account()
IBMQ.providers()

In [ ]:
provider = IBMQ.get_provider("INSERT PROVIDER")

provider.backends()

## The QAOA function

In [9]:
def testing_quantum(ins, backend, optimal_point, spsa_trials, trials, p_value):
    """
    Input:
    ins: name of instance
    backend
    optimal_point
    spsa_trials
    trials
    
    Output:
    outmatrix : [soln,result time,eigenvector,number of spsa trials]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    machinesolutions = []
    rmachinesolutions = []
    seed = 10598
    mod = QuadraticProgram()
    mod.from_docplex(mdl)


    #trials
    ans = np.zeros(5)
    file = open("thirty_trials-" + str(backend) + "-" + str(ins[-9:]) ,"w")
    file.write("value,   feasible,   frequency, time, iteration" + "\n")
    for i in range(trials):
        spsa = SPSA(maxiter=spsa_trials) #classical optimiser
        
        Backend = provider.get_backend(backend) #backend
        quantum_instance = QuantumInstance(backend=Backend,seed_simulator=seed,
                                           seed_transpiler=seed, skip_qobj_validation = False, shots = 8192)

        qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=p_value,include_custom=True,initial_point = optimal_point)
        result = qaoa.run(quantum_instance) #results
        
        
        print('QAOA optimal point:', result['optimal_point'])
        print('QAOA time:', result['optimizer_time'])
        print('QAOA QAP objective:', result['eigenvalue'] + offset_docplex)
        
        #print results csv-files
        n = len(list(result['eigenstate'].values()))
        solution = np.hstack((np.array(list(result['eigenstate'].values())).reshape(n,1),np.array(list(result['eigenstate'].keys())).reshape(n,1)))
        print(solution)
        for r in solution:
            file.write(str(qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc)) + "," + str(qap_feasible(np.array(list(r[1])))) + "," + str(r[0]) + ","+ str(result['optimizer_time']) + "," + str(i) + "\n")
            ans = np.vstack((ans,np.array([qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc),qap_feasible(np.array(list(r[1]))),float(r[0]),result['optimizer_time'],i])))
        except:
            file.write("An error occurred on iteration " + str(i) + " of the " + str(trials) + " trials")
            print("AN ERROR OCCURRED ON ITERATION" + str(i) + " of the " + str(trials) + " trials")
            ans = np.vstack((ans,np.array([math.inf,False,0,np.nan,i])))
        print("Iteration "+ str(i) + " is complete.")
    file.close()
    return ans[1:] 
    

In [10]:
def read_optimal(ins):
    data = pd.read_csv("initial_point-" + str(ins),header = None)
    n = len(data[0]) - 1
    ans = []
    data[0][0] = data[0][0][1:]
    data[0][n] = data[0][n][:-1]
    for i in data[0]:
        r = i.split(" ")
        for t in r:
            if t!='':
                ans.append(float(t))
    return np.array(ans)

## The results cells

In [ ]:
ins = matrix_size_3
#import logging
#logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend = 'ibm_cairo'
optimal_point = read_optimal(ins)
p = 1
ans = testing_quantum(ins, backend, optimal_point, 1, 30, p)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright